In [1]:
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
import sys
import requests
# create spark configuration
conf = SparkConf()
conf.setAppName("twitterStream")

In [2]:
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")

In [3]:
ssc = StreamingContext(sc, 10)

In [4]:
ssc.checkpoint("checkpoint_TwitterApp")

In [5]:
dataStream = ssc.socketTextStream("localhost",5045)

In [7]:
def aggregate_tags_count(new_values, total_sum):
    return sum(new_values) + (total_sum or 0)

In [8]:
def get_sql_context_instance(spark_context):
    if ('sqlContextSingletonInstance' not in globals()):
        globals()['sqlContextSingletonInstance'] = SQLContext(spark_context)
    return globals()['sqlContextSingletonInstance']

In [10]:
def process_rdd(time, rdd):
    print("----------- %s -----------" % str(time))
    try:
        #Get spark sql singleton context from the current context
        sql_context = get_sql_context_instance(rdd.context)
        # convert the RDD to Row RDD
        row_rdd = rdd.map(lambda w: Row(hashtag=w[0], hashtag_count=w[1]))
        # create a DF from the Row RDD
        hashtags_df = sql_context.createDataFrame(row_rdd)
        # Register the dataframe as table
        hashtags_df.registerTempTable("hashtags")
        # get the top 10 hashtags from the table using SQL and print them
        hashtag_counts_df = sql_context.sql("select hashtag, hashtag_count from hashtags order by hashtag_count desc limit 10")
        hashtag_counts_df.show()
    except:
        e= sys.exc_info()[0]
        print("Error: %s" % e)

In [12]:
words = dataStream.flatMap(lambda line: line.split(" "))

In [13]:
hashtags = words.filter(lambda w: '#' in w).map(lambda x: (x, 1))

In [14]:
tags_totals = hashtags.updateStateByKey(aggregate_tags_count)

In [15]:
# do processing for each RDD generated in each interval
tags_totals.foreachRDD(process_rdd)


In [16]:
dataStream.pprint()

In [17]:
ssc.start()
ssc.awaitTermination()

----------- 2021-04-13 09:54:40 -----------
Error: <class 'ValueError'>
-------------------------------------------
Time: 2021-04-13 09:54:40
-------------------------------------------
@TweetTylerOrr Omg lol quality time is how I feel loved and acts of service is how I show love. You’re so good at this game 😂

----------- 2021-04-13 09:54:50 -----------
Error: <class 'ValueError'>
-------------------------------------------
Time: 2021-04-13 09:54:50
-------------------------------------------
@skye_daddy No, I’m both proud and grateful for being able to find a way up and for there being a path to begin wit… https://t.co/lQffVWxnN3

----------- 2021-04-13 09:55:00 -----------
Error: <class 'ValueError'>
-------------------------------------------
Time: 2021-04-13 09:55:00
-------------------------------------------
no THEY understood the assignment https://t.co/CLf3L1rlBp
@Heidiaca That’s me in heels 😂😂

----------- 2021-04-13 09:55:10 -----------
Error: <class 'ValueError'>
----------

KeyboardInterrupt: 

----------- 2021-04-13 10:11:30 -----------
+--------------------+-------------+
|             hashtag|hashtag_count|
+--------------------+-------------+
|           #GoEagles|            1|
|        #ElmerGantry|            1|
|              #Blues|            1|
|         #SMMHSsoars|            1|
|                 #IT|            1|
|        #comfortfood|            1|
|    #LiveStreamMusic|            1|
|         #Nashville,|            1|
|                 #RN|            1|
|THIS.👇🏾👇🏾👇🏾...|            1|
+--------------------+-------------+

-------------------------------------------
Time: 2021-04-13 10:11:30
-------------------------------------------

----------- 2021-04-13 10:11:40 -----------
+--------------------+-------------+
|             hashtag|hashtag_count|
+--------------------+-------------+
|           #GoEagles|            1|
|        #ElmerGantry|            1|
|              #Blues|            1|
|         #SMMHSsoars|            1|
|                 #IT|   